In [3]:
!pip install tensorflow
!pip install keras

In [4]:
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, KFold
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.optimizers import Adam
from keras.layers import Dropout

In [6]:
turbine_data=pd.read_csv("gas_turbines.csv")

In [7]:
turbine_data

,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY,CDP,CO,NOX
0,6.8594,1007.9,96.799,3.5000,19.663,1059.2,550.00,114.70,10.605,3.1547,82.722
1,6.7850,1008.4,97.118,3.4998,19.728,1059.3,550.00,114.72,10.598,3.2363,82.776
2,6.8977,1008.8,95.939,3.4824,19.779,1059.4,549.87,114.71,10.601,3.2012,82.468
3,7.0569,1009.2,95.249,3.4805,19.792,1059.6,549.99,114.72,10.606,3.1923,82.670
4,7.3978,1009.7,95.150,3.4976,19.765,1059.7,549.98,114.72,10.612,3.2484,82.311
...,...,...,...,...,...,...,...,...,...,...,...
15034,9.0301,1005.6,98.460,3.5421,19.164,1049.7,546.21,111.61,10.400,4.5186,79.559
15035,7.8879,1005.9,99.093,3.5059,19.414,1046.3,543.22,111.78,10.433,4.8470,79.917
15036,7.2647,1006.3,99.496,3.4770,19.530,1037.7,537.32,110.19,10.483,7.9632,90.912
15037,7.0060,1006.8,99.008,3.4486,19.377,1043.2,541.24,110.74,10.533,6.2494,93.227


In [8]:
turbine_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15039 entries, 0 to 15038
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AT      15039 non-null  float64
 1   AP      15039 non-null  float64
 2   AH      15039 non-null  float64
 3   AFDP    15039 non-null  float64
 4   GTEP    15039 non-null  float64
 5   TIT     15039 non-null  float64
 6   TAT     15039 non-null  float64
 7   TEY     15039 non-null  float64
 8   CDP     15039 non-null  float64
 9   CO      15039 non-null  float64
 10  NOX     15039 non-null  float64
dtypes: float64(11)
memory usage: 1.3 MB


In [9]:
turbine_data.isna().sum()

AT      0
AP      0
AH      0
AFDP    0
GTEP    0
TIT     0
TAT     0
TEY     0
CDP     0
CO      0
NOX     0
dtype: int64

In [10]:
turbine_data.describe()

,AT,AP,AH,AFDP,GTEP,TIT,TAT,TEY,CDP,CO,NOX
count,15039.000000,15039.00000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000,15039.000000
mean,17.764381,1013.19924,79.124174,4.200294,25.419061,1083.798770,545.396183,134.188464,12.102353,1.972499,68.190934
std,7.574323,6.41076,13.793439,0.760197,4.173916,16.527806,7.866803,15.829717,1.103196,2.222206,10.470586
min,0.522300,985.85000,30.344000,2.087400,17.878000,1000.800000,512.450000,100.170000,9.904400,0.000388,27.765000
25%,11.408000,1008.90000,69.750000,3.723900,23.294000,1079.600000,542.170000,127.985000,11.622000,0.858055,61.303500
50%,18.186000,1012.80000,82.266000,4.186200,25.082000,1088.700000,549.890000,133.780000,12.025000,1.390200,66.601000
75%,23.862500,1016.90000,90.043500,4.550900,27.184000,1096.000000,550.060000,140.895000,12.578000,2.160400,73.935500
max,34.929000,1034.20000,100.200000,7.610600,37.402000,1100.800000,550.610000,174.610000,15.081000,44.103000,119.890000


In [11]:
turbine_data.TEY.mean()

134.18846399361655

In [12]:
turbine_data.loc[turbine_data["TEY"]<=134, 'TEY_new']='low'
turbine_data.loc[turbine_data["TEY"]>=134, 'TEY_new']='high'

In [13]:
turbine_data = turbine_data.drop(['TEY'], axis=1)

In [14]:
turbine_data

,AT,AP,AH,AFDP,GTEP,TIT,TAT,CDP,CO,NOX,TEY_new
0,6.8594,1007.9,96.799,3.5000,19.663,1059.2,550.00,10.605,3.1547,82.722,low
1,6.7850,1008.4,97.118,3.4998,19.728,1059.3,550.00,10.598,3.2363,82.776,low
2,6.8977,1008.8,95.939,3.4824,19.779,1059.4,549.87,10.601,3.2012,82.468,low
3,7.0569,1009.2,95.249,3.4805,19.792,1059.6,549.99,10.606,3.1923,82.670,low
4,7.3978,1009.7,95.150,3.4976,19.765,1059.7,549.98,10.612,3.2484,82.311,low
...,...,...,...,...,...,...,...,...,...,...,...
15034,9.0301,1005.6,98.460,3.5421,19.164,1049.7,546.21,10.400,4.5186,79.559,low
15035,7.8879,1005.9,99.093,3.5059,19.414,1046.3,543.22,10.433,4.8470,79.917,low
15036,7.2647,1006.3,99.496,3.4770,19.530,1037.7,537.32,10.483,7.9632,90.912,low
15037,7.0060,1006.8,99.008,3.4486,19.377,1043.2,541.24,10.533,6.2494,93.227,low


In [15]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
def label_encoding(data):
    for column_name in data.columns:
        if data[column_name].dtype==object:
            data[column_name]= le.fit_transform(data[column_name])
        else:
            pass


In [16]:
label_encoding(turbine_data)

In [17]:
turbine_data

,AT,AP,AH,AFDP,GTEP,TIT,TAT,CDP,CO,NOX,TEY_new
0,6.8594,1007.9,96.799,3.5000,19.663,1059.2,550.00,10.605,3.1547,82.722,1
1,6.7850,1008.4,97.118,3.4998,19.728,1059.3,550.00,10.598,3.2363,82.776,1
2,6.8977,1008.8,95.939,3.4824,19.779,1059.4,549.87,10.601,3.2012,82.468,1
3,7.0569,1009.2,95.249,3.4805,19.792,1059.6,549.99,10.606,3.1923,82.670,1
4,7.3978,1009.7,95.150,3.4976,19.765,1059.7,549.98,10.612,3.2484,82.311,1
...,...,...,...,...,...,...,...,...,...,...,...
15034,9.0301,1005.6,98.460,3.5421,19.164,1049.7,546.21,10.400,4.5186,79.559,1
15035,7.8879,1005.9,99.093,3.5059,19.414,1046.3,543.22,10.433,4.8470,79.917,1
15036,7.2647,1006.3,99.496,3.4770,19.530,1037.7,537.32,10.483,7.9632,90.912,1
15037,7.0060,1006.8,99.008,3.4486,19.377,1043.2,541.24,10.533,6.2494,93.227,1


In [18]:
def norm_func(i):
    x = (i-i.min())/(i.max()-i.min())
    return (x)

In [19]:
turbine_data= norm_func(turbine_data)

In [20]:
X = turbine_data.iloc[:, :-1]
Y = turbine_data.iloc[: , -1]

In [21]:
X

,AT,AP,AH,AFDP,GTEP,TIT,TAT,CDP,CO,NOX
0,0.184182,0.456050,0.951314,0.255758,0.091426,0.584,0.984015,0.135340,0.071522,0.596548
1,0.182020,0.466391,0.955881,0.255721,0.094755,0.585,0.984015,0.133988,0.073372,0.597134
2,0.185295,0.474664,0.939003,0.252571,0.097367,0.586,0.980608,0.134567,0.072576,0.593791
3,0.189922,0.482937,0.929126,0.252227,0.098033,0.588,0.983753,0.135533,0.072375,0.595984
4,0.199830,0.493278,0.927708,0.255323,0.096650,0.589,0.983491,0.136692,0.073647,0.592087
...,...,...,...,...,...,...,...,...,...,...
15034,0.247272,0.408480,0.975092,0.263380,0.065868,0.489,0.884696,0.095739,0.102448,0.562214
15035,0.214075,0.414685,0.984153,0.256826,0.078672,0.455,0.806342,0.102113,0.109894,0.566100
15036,0.195962,0.422958,0.989922,0.251593,0.084614,0.369,0.651730,0.111772,0.180552,0.685449
15037,0.188443,0.433299,0.982936,0.246451,0.076777,0.424,0.754455,0.121431,0.141693,0.710578


In [22]:
def creat_model():
    model = Sequential()
    model.add(Dense(14, input_dim=10, kernel_initializer='uniform',activation='relu'))
    model.add(Dense(8, kernel_initializer='uniform', activation='relu'))
    model.add(Dense(1, kernel_initializer='uniform',activation='sigmoid'))
    
    adam=Adam(learning_rate=0.01)
    model.compile(loss='mean_squared_error', optimizer=adam, metrics=['accuracy'])
    return model

In [23]:
model = KerasClassifier(build_fn= creat_model, verbose=0)
# Define the grid search parameter
batch_size=[10,20,40]
epochs= [10,50,100]
# make s directory of grid search parameters
param_grid= dict(batch_size=batch_size, epochs=epochs)
# Build and fit the GridSearchCV
grid= GridSearchCV(estimator=model, param_grid=param_grid, cv = KFold(), verbose=10)
grid_result= grid.fit(X,Y)

C:\Users\BHAVYA~1.N\AppData\Local\Temp/ipykernel_42636/3540770385.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn= creat_model, verbose=0)


Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START batch_size=10, epochs=10....................................
[CV 1/5; 1/9] END .....batch_size=10, epochs=10;, score=0.926 total time=  25.2s
[CV 2/5; 1/9] START batch_size=10, epochs=10....................................
[CV 2/5; 1/9] END .....batch_size=10, epochs=10;, score=0.756 total time=  24.3s
[CV 3/5; 1/9] START batch_size=10, epochs=10....................................
[CV 3/5; 1/9] END .....batch_size=10, epochs=10;, score=0.838 total time=  20.7s
[CV 4/5; 1/9] START batch_size=10, epochs=10....................................
[CV 4/5; 1/9] END .....batch_size=10, epochs=10;, score=0.916 total time=  19.6s
[CV 5/5; 1/9] START batch_size=10, epochs=10....................................
[CV 5/5; 1/9] END .....batch_size=10, epochs=10;, score=0.896 total time=  19.4s
[CV 1/5; 2/9] START batch_size=10, epochs=50....................................
[CV 1/5; 2/9] END .....batch_size=10, epochs=50;,

In [24]:
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.909569263458252, using {'batch_size': 40, 'epochs': 100}
0.8662164688110352,0.06315239409849212 with: {'batch_size': 10, 'epochs': 10}
0.8867634177207947,0.05680243690128851 with: {'batch_size': 10, 'epochs': 50}
0.827122163772583,0.15473540348031797 with: {'batch_size': 10, 'epochs': 100}
0.8908829689025879,0.02965255496226401 with: {'batch_size': 20, 'epochs': 10}
0.8880929827690125,0.040981999946672 with: {'batch_size': 20, 'epochs': 50}
0.8819752097129822,0.035501787288584025 with: {'batch_size': 20, 'epochs': 100}
0.8819741249084473,0.025695413167252018 with: {'batch_size': 40, 'epochs': 10}
0.8970020532608032,0.03463941245763555 with: {'batch_size': 40, 'epochs': 50}
0.909569263458252,0.03046575730180719 with: {'batch_size': 40, 'epochs': 100}


In [25]:
from keras.layers import Dropout

In [26]:
def create_model(learning_rate,dropout_rate):
    model = Sequential()
    model.add(Dense(14,input_dim = 10,kernel_initializer = 'normal',activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(8,input_dim = 14,kernel_initializer = 'normal',activation = 'relu'))
    model.add(Dropout(dropout_rate))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(learning_rate = learning_rate)
    model.compile(loss = 'mean_absolute_error',optimizer = adam,metrics = ['accuracy'])
    return model

In [27]:
model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 100)

C:\Users\BHAVYA~1.N\AppData\Local\Temp/ipykernel_42636/2763922832.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 100)


In [28]:
learning_rate = [0.001,0.01,0.1]
dropout_rate = [0.0,0.1,0.2]

In [29]:
param_grids = dict(learning_rate = learning_rate,dropout_rate = dropout_rate)

In [30]:
grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X,Y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 1/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=0.922 total time= 1.0min
[CV 2/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 2/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=0.871 total time=  56.8s
[CV 3/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 3/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=0.801 total time=  51.0s
[CV 4/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 4/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=0.932 total time=  49.2s
[CV 5/5; 1/9] START dropout_rate=0.0, learning_rate=0.001.......................
[CV 5/5; 1/9] END dropout_rate=0.0, learning_rate=0.001;, score=0.920 total time=  49.3s
[CV 1/5; 2/9] START dropout_rate=0.0, learning_rate=0.01........................
[CV 1/5; 

In [31]:
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.8992617845535278, using {'dropout_rate': 0.1, 'learning_rate': 0.001}
0.8888909220695496,0.04894220344895828 with: {'dropout_rate': 0.0, 'learning_rate': 0.001}
0.8351616621017456,0.04938567152120312 with: {'dropout_rate': 0.0, 'learning_rate': 0.01}
0.6427915871143342,0.13413603682838834 with: {'dropout_rate': 0.0, 'learning_rate': 0.1}
0.8992617845535278,0.029028232951922035 with: {'dropout_rate': 0.1, 'learning_rate': 0.001}
0.8480665445327759,0.07003595437952756 with: {'dropout_rate': 0.1, 'learning_rate': 0.01}
0.7670741558074952,0.05091413393524975 with: {'dropout_rate': 0.1, 'learning_rate': 0.1}
0.8921486377716065,0.04210969685600925 with: {'dropout_rate': 0.2, 'learning_rate': 0.001}
0.8435425162315369,0.05789449062230908 with: {'dropout_rate': 0.2, 'learning_rate': 0.01}
0.686091810464859,0.1435292316847027 with: {'dropout_rate': 0.2, 'learning_rate': 0.1}


In [32]:
def create_model(activation_function,init):
    model = Sequential()
    model.add(Dense(14,input_dim = 10,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(0.0))
    model.add(Dense(8,input_dim = 14,kernel_initializer = init,activation = activation_function))
    model.add(Dropout(0.0))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(learning_rate = 0.01)
    model.compile(loss = 'mean_absolute_error',optimizer = adam,metrics = ['accuracy'])
    return model

In [33]:
model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 100)

C:\Users\BHAVYA~1.N\AppData\Local\Temp/ipykernel_42636/2763922832.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 100)


In [34]:
activation_function = ['softmax','relu','tanh','linear']
init = ['uniform','normal','zero']

In [35]:
param_grids = dict(activation_function = activation_function,init = init)

In [36]:
grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X,Y)

Fitting 5 folds for each of 12 candidates, totalling 60 fits
[CV 1/5; 1/12] START activation_function=softmax, init=uniform..................
[CV 1/5; 1/12] END activation_function=softmax, init=uniform;, score=0.932 total time= 1.0min
[CV 2/5; 1/12] START activation_function=softmax, init=uniform..................
[CV 2/5; 1/12] END activation_function=softmax, init=uniform;, score=0.774 total time=  51.9s
[CV 3/5; 1/12] START activation_function=softmax, init=uniform..................
[CV 3/5; 1/12] END activation_function=softmax, init=uniform;, score=0.865 total time=  57.4s
[CV 4/5; 1/12] START activation_function=softmax, init=uniform..................
[CV 4/5; 1/12] END activation_function=softmax, init=uniform;, score=0.932 total time=  57.2s
[CV 5/5; 1/12] START activation_function=softmax, init=uniform..................
[CV 5/5; 1/12] END activation_function=softmax, init=uniform;, score=0.926 total time=  52.4s
[CV 1/5; 2/12] START activation_function=softmax, init=normal...

In [38]:
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.9092363834381103, using {'activation_function': 'softmax', 'init': 'normal'}
0.8858328580856323,0.06129641579392881 with: {'activation_function': 'softmax', 'init': 'uniform'}
0.9092363834381103,0.025528346351838586 with: {'activation_function': 'softmax', 'init': 'normal'}
0.8803112745285034,0.023320185091070655 with: {'activation_function': 'softmax', 'init': 'zero'}
0.7908807277679444,0.11708952003857546 with: {'activation_function': 'relu', 'init': 'uniform'}
0.8266509056091309,0.06232362327890864 with: {'activation_function': 'relu', 'init': 'normal'}
0.5625389397144318,0.06284089125503536 with: {'activation_function': 'relu', 'init': 'zero'}
0.799721610546112,0.08078528103568501 with: {'activation_function': 'tanh', 'init': 'uniform'}
0.8113585829734802,0.0547610434325418 with: {'activation_function': 'tanh', 'init': 'normal'}
0.5625389397144318,0.06284089125503536 with: {'activation_function': 'tanh', 'init': 'zero'}
0.8124219894409179,0.038898227324800694 with: {'activ

In [39]:
def create_model(neuron1,neuron2):
    model = Sequential()
    model.add(Dense(neuron1,input_dim = 10,kernel_initializer = 'normal',activation = 'relu'))
    model.add(Dropout(0.0))
    model.add(Dense(neuron2,input_dim = neuron1,kernel_initializer = 'normal',activation = 'relu'))
    model.add(Dropout(0.0))
    model.add(Dense(1,activation = 'sigmoid'))
    
    adam = Adam(learning_rate = 0.01)
    model.compile(loss = 'mean_absolute_error',optimizer = adam,metrics = ['accuracy'])
    return model

In [40]:
model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 100)

C:\Users\BHAVYA~1.N\AppData\Local\Temp/ipykernel_42636/2763922832.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 100)


In [41]:
neuron1 = [4,8,16]
neuron2 = [2,4,8]

In [42]:
param_grids = dict(neuron1 = neuron1,neuron2 = neuron2)

In [43]:
grid = GridSearchCV(estimator = model,param_grid = param_grids,cv = KFold(),verbose = 10)
grid_result = grid.fit(X,Y)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 1/5; 1/9] END .........neuron1=4, neuron2=2;, score=0.521 total time=  40.2s
[CV 2/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 2/5; 1/9] END .........neuron1=4, neuron2=2;, score=0.670 total time=  38.6s
[CV 3/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 3/5; 1/9] END .........neuron1=4, neuron2=2;, score=0.812 total time=  37.7s
[CV 4/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 4/5; 1/9] END .........neuron1=4, neuron2=2;, score=0.549 total time=  36.9s
[CV 5/5; 1/9] START neuron1=4, neuron2=2........................................
[CV 5/5; 1/9] END .........neuron1=4, neuron2=2;, score=0.586 total time=  41.7s
[CV 1/5; 2/9] START neuron1=4, neuron2=4........................................
[CV 1/5; 2/9] END .........neuron1=4, neuron2=4;,

In [44]:
print('Best : {}, using {}'.format(grid_result.best_score_,grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print('{},{} with: {}'.format(mean, stdev, param))

Best : 0.868609607219696, using {'neuron1': 16, 'neuron2': 2}
0.6275655269622803,0.10508675139272275 with: {'neuron1': 4, 'neuron2': 2}
0.8668821215629577,0.03201059540667445 with: {'neuron1': 4, 'neuron2': 4}
0.8428080201148986,0.09558120489342106 with: {'neuron1': 4, 'neuron2': 8}
0.6815549015998841,0.12595459243641705 with: {'neuron1': 8, 'neuron2': 2}
0.8559736728668212,0.06257456863316649 with: {'neuron1': 8, 'neuron2': 4}
0.8399489164352417,0.04829788289982147 with: {'neuron1': 8, 'neuron2': 8}
0.868609607219696,0.024859099046730247 with: {'neuron1': 16, 'neuron2': 2}
0.8193346738815308,0.06145453701678219 with: {'neuron1': 16, 'neuron2': 4}
0.852781879901886,0.05082321998613478 with: {'neuron1': 16, 'neuron2': 8}


In [45]:
from sklearn.metrics import classification_report, accuracy_score

In [46]:
def create_model():
    model = Sequential()
    model.add(Dense(8,input_dim = 10,kernel_initializer = 'normal',activation = 'relu'))
    model.add(Dropout(0.0))
    model.add(Dense(8,input_dim = 8,kernel_initializer = 'normal',activation = 'relu'))
    model.add(Dropout(0.0))
    model.add(Dense(1,activation = 'relu'))
    
    adam = Adam(learning_rate = 0.01) #sgd = SGD(lr=learning_rate, momentum=momentum, decay=decay_rate, nesterov=False)
    model.compile(loss = 'mean_absolute_error',optimizer = adam,metrics = ['accuracy'])
    return model

In [47]:
model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 100)

C:\Users\BHAVYA~1.N\AppData\Local\Temp/ipykernel_42636/2763922832.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 100)


In [48]:
model.fit(X,Y)

In [49]:
y_predict = model.predict(X)

In [50]:
print(accuracy_score( Y ,y_predict))

0.4374625972471574


In [51]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, random_state=40)

In [52]:
from sklearn.metrics import classification_report, accuracy_score
from keras.layers import Dropout

In [53]:
def create_model():
    model = Sequential()
    model.add(Dense(8,input_dim = 10,kernel_initializer = 'normal',activation = 'relu'))
    model.add(Dropout(0.0))
    model.add(Dense(8,input_dim = 8,kernel_initializer = 'normal',activation = 'relu'))
    model.add(Dropout(0.0))
    model.add(Dense(1,activation = 'relu'))
    
    adam = Adam(learning_rate = 0.01) 
    model.compile(loss = 'mean_absolute_error',optimizer = adam,metrics = ['accuracy'])
    return model

In [54]:
model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 100)

C:\Users\BHAVYA~1.N\AppData\Local\Temp/ipykernel_42636/2763922832.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  model = KerasClassifier(build_fn = create_model,verbose = 0,batch_size = 40,epochs = 100)


In [55]:
model.fit(x_train,y_train)

In [56]:
y_predict = model.predict(x_test)

In [57]:
print(accuracy_score( y_test ,y_predict))

0.4408244680851064
